In [66]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
from scipy.optimize import fsolve

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [109]:
## MAKE RIKS DO NOT ERASE!!! 

#specs = loaders.load_density()
#specs = geography.get_coordinate_functions(specs)
#airports = loaders.load_airports(specs)
#travel = loaders.load_travel(airports)['annual']
print('finish loading')

wuhan_R0 = 4
region = 'global'
kappa = 1
infected = 1
wuhan_factor =  wuhan_R0 / airports.loc['WUH', 'density'] # R_0 / density for Wuhan

airports['R0'] = airports.density * wuhan_factor
airports['p_outbreak'] = loaders.calculate_outbreaks(airports, kappa=kappa, n=infected)
travel = loaders.augment_travel(travel, airports) 
riskiest = travel.query('Origin == "WUH"')[['Dest', 'risk_ij']].sort_values('risk_ij').iloc[-10:]
riskiest = dict(zip(riskiest.Dest, riskiest.risk_ij))
df = pd.read_csv('./data/airports.csv')
df = df.query('iata_code in @riskiest.keys()')
drop = ['latitude_deg', 'elevation_ft', 'longitude_deg', 
        'scheduled_service', 'type',  'id', 'home_link', 
        'local_code', 'gps_code', 'wikipedia_link', 'keywords',
        'iso_country', 'continent', 'iso_region', 'ident']
df = df.assign(risk=df.iata_code.replace(riskiest)).drop(drop, axis=1, errors='ignore')
df = df.sort_values('risk',ascending=False).set_index('municipality', drop=True)
df.loc['Beijing', 'infection'] = pd.Timestamp(day=19, month=1, year=2020)
#df.loc['Guandgdong', 'infection'] = pd.Timestamp(day=19, month=1, year=2020)
df.loc['Shanghai', 'infection'] = pd.Timestamp(day=20, month=1, year=2020)
df.loc['Shenzhen', 'infection'] = pd.Timestamp(day=10, month=1, year=2020)
df.loc['Tianjin', 'infection'] = pd.Timestamp(day=22, month=1, year=2020)
df.loc['Chongqing', 'infection'] = pd.Timestamp(day=21, month=1, year=2020)
df.to_csv(f'riskiest_wuhan{wuhan_R0}.csv')
df

finish loading


,name,iata_code,risk,infection
municipality,,,,
Beijing,Beijing Capital International Airport,PEK,0.096605,2020-01-19
Guangzhou,Guangzhou Baiyun International Airport,CAN,0.074078,NaT
Shanghai,Shanghai Pudong International Airport,PVG,0.067223,2020-01-20
Shenzhen,Shenzhen Bao'an International Airport,SZX,0.035457,2020-01-10
Chengdu,Chengdu Shuangliu International Airport,CTU,0.035407,NaT
Tianjin,Tianjin Binhai International Airport,TSN,0.027307,2020-01-22
Xi'an,Xi'an Xianyang International Airport,XIY,0.020167,NaT
Xiamen,Xiamen Gaoqi International Airport,XMN,0.017157,NaT
Kunming,Kunming Changshui International Airport,KMG,0.014993,NaT
